# Analysing Spatial Data with PostGIS

Spatial data is a critical piece of the data ecosystem. A phone app can find nearby coffee shops because it queries a spatial database, asking it to return a list of shops within a certain distance of our location. Governments use spatial data to track the footprints of residential & business parcels; epidemiologists use it to visualise the spread of diseases.

---

# Enabling PostGIS & Creating a Spatial Database

PostGIS is a free, open source project created by the Canadian geospatial company Refractions Research & maintained by an international team of developers under the Open Source Geospatial Foundation (OSGeo). The GIS portion of its name refers to *geographic information system*, defined as a system that allows for storing, editing, analysing, & displaying spatial data.

To enable PostGIS on the `analysis` database, open pgAdmin's Query Tool & run the statement below.

```
CREATE EXTENSION postgis;
```

You'll see the message `CREATE EXTENSION`, advising that your database has been updated to include spatial data types & analysis functions. Run `SELECT postgis_full_version();` to display the version number of PostGIS along with the versions of its installed components.

---

# Understanding the Building Blocks of Spatial Data

A point on a grid is the smallest building block of spatial data. The grid might be marked with x- & y- axes, or longitude & latitude if we're using a map. A grid could be flat with two dimensions, or it could describe a three-dimensional space such as a cube. In some data formats, such as the JavaScript-based *GeoJSON*, a point may have attributes in addition to its location. We could describe a grocery store with a point containing its longitude & latitude as well as attributes for the store's name & hours of operation.

---

# Understanding Two-Dimensional Geometrics

The Open Geospatial Consortium (OGC) & International Organisation for Standardisation (ISO) have created a *simple features access* model that describes standards for building & querying two- & three-dimensional shapes, sometimes refered to as *geometries*. PostGIS supports the standard.

The following are the more common features, starting with points & building in complexity:

* **Point**
   - A single location in a two- or three-dimensional plane. On maps, a Point is usually a dot marking a longitude & latitude.
* **LineString**
   - Two or more Points, each connected by straight lines. A LineString can represent features such as a road, biking trail, or stream.
* **Polygon**
   - A two-dimensional shape with three or more straight sides, each constructed from a LineString. On maps, Polygons represent objects such as nations, states, buildings, & bodies of water. A Polygon can have one or more Polygons that act as holes inside a larger Polygon.
* **MultiPoint**
   - A set of Points. A single MultiPoint object could represent multiple locations of a retail with each store's latitude & longitude.
* **MultiLineString**
   - A set of LineStrings. An example is a road that has several noncontinuous segments.
* **MultiPolygon**
   - A set of Polygons. A parcel of land that's divided into parts by a road could be grouped in one MultiPolygon object instead of separated polygons.
 
The figure below shows an example of each feature.

<img src = "Visual Examples of Geometries.png" width = "500" style = "margin:auto"/>

PostGIS enables functions to build, edit, & analyse these objects. These functions take a variety of inputs depending on their purpose, including latitude & longitude, specialised text & binary formats, & simple features. Some functions also take an optional *spatial reference system identifier* (SRID) that specifies the grid on which to place the objects.

## Well-Known Text Formats

The OGC standard's WKT format specifies a geometry type & its coordinates inside one or more sets of parentheses. The number of coordinates & parentheses varies depending on the type of geometry. The table below shows examples of frequently use geometry types & their WKT formats. Longitude/latitude pairs are shown for the coordinates, but you might encounter grid systems that use other measures.

|Geometry|Format|Notes|
|:---|:---|:---|
|Point|`POINT (-74.9, 42.7)`|A coordinate pair marking a point at -74.9 longitude & 42.7 latitude.|
|LineString|LINESTRING (-74.9 42.7, -75.1 42.7)|A straight line with endpoints marked by two coordinate pairs.|
|Polygon|`POLYGON ((-74.9 42.7, -75.1 42.7, -75.1 42.6, -74.9 42.7))`|A triangle outlined by three different pairs of coordinates. Although listed twice, the first & last pair are the same coordinates where we close the shape.|
|MultiPoint|`MULTIPOINT (-74.9 42.7, -75.1 42.7)`|Two Points, one for each pair of coordinates.|
|MultiLineString|`MULTILINESTRING ((-76.27 43.1, -76.06 43.08), (-76.2 43.3, -76.2 43.4, -76.4 43.1))`|Two LineStrings. The first has two points; the second has three.|
|MultiPolygon|`MULTIPOLYGON (((-74.94 42.7, -75.06 42.71, -75.07 42.64, -75.92 42.7), (-75.0 42.66, -75.0 42.64, -74.98 42.64, -74.98 42.66, -75.0 42.66)))`|Two Polygons. The first is a triangle, & the second is a rectangle.|

## Projections & Coordinate Systems

Representing Earth's spherical surface on a two-dimensional map is not easy. Thus, what cartographers do is create a map *projection* on its own *projected coordinate system*. A projection is simply a flattened representation of the globe with its own two-dimensional coordinate system.

Projections are derived from *geographic coordinate systems*, which define the grid of latitude, longitude, & height of any point on the globe along with factors including Earth's shape. Whenever you obtain geographic data, it's critical to know the coordinate systems it references so you provide the correct information when writing queries. Often, user documentation will name the coordinate system.

## Spatial Reference System Identifier

When using PostGIS (& other GIS applications), you specify the coordinate system via its unique SRID. When you enabled the PostGIS extension, the process created the table `spatial_ref_sys`, which contains SRIDs as its primary key. The table also contains the column `srtext`, which includes a WKT representation of the spatial reference system plus other metadata.

We'll be using SRID `4326`, the ID for the geographic coordinate system WGS 84. That's the most recent World Geodetic System (WGS) standard used by GPS. You can see the WKT representation for WGS 84 by running the query below:

```
SELECT srtext
FROM spatial_ref_sys
WHERE srid = 4326;
```

You should get the following result.

<img src = "Retrieving the WKT for SRID 4326.png" width = "600" style = "margin:auto"/>

We won't need to use this information for this lesson's exercises, but we can get to know some of the variables & how the define the projection. `GEOGCS` provides the geographic coordinate system in use. `PRIMEM` specifies the location of the *prime meridian*, or longitude 0.

Conversely, if you ever need to find the SRID associated with a coordinate system, you can query the `srtext` column in `spatial_ref_sys` to find it.

---

# Understanding PostGIS Data Types

Installing PostGIS adds several data types to your database. We'll use two: `geography` & `geometry`. Both types can store spatial data, such as the points, lines, polygons, & SRIDs, but they have important distinctions:

* **geography:** A data type based on a sphere, using the round-Earth coordinate system (longitude & latitude). All calculations occur on the globe, taking its curvature into account. This makes the math complex & limits the number of functions available to work with the `geography` type. but because Earth's curvature is factored in, calculations for distance are more precies; you should use the `geography` data type when handling data that spans large areas. The results from calculations on the `geography` type will be expressed in meters.
* **geometry:** A data type based on a plane, using the Euclidean coordinate system. Calculations occur on straight lines as opposed to along the curvature of a sphere, making calculations for geographical distance less precise than with the `geography` data type; the results of calculations are expressed in units of whichever coordinate system you've designated.

The [PostGIS documentation](https://postgis.net/docs/using_postgis_dbmanagement.html) offers guidance on when to use one or the other type. In short, if you're working strictly with longitude/latitude data or if your data covers a large area, such as a continent or globe, use the `geography` type, even though it limits the functions you can use. If your data covers a smaller area, the `geometry` type provides more functions & better performance. You can also convert one type to the other using `CAST`.

---

# Creating Spatial Objects with PostGIS Functions

PostGIS has more than three dozen constructor [functions](https://postgis.net/docs/reference.html#Geometry_Constructors) that build spatial objects using WKT or coordinates. Most PostGIS functions begin with the letters *ST*, which is an ISO naming standard that means *spatial type*.

## Creating a Geometry Type from Well-Known Text

The `ST_GeomFromText(WKT, SRID)` function creates a `geometry` data type from an input of a WKT string & an optional SRID. The queries below shows simple `SELECT` statements that generate `geometry` data types for each of the WKT text formats in the table above.

```
SELECT ST_GeomFromText(
           'POINT(-74.9233606 42.699992)', 4326);

SELECT ST_GeomFromText(
           'LINESTRING(-74.9 42.7, -75.1 42.7)',
           4326);

SELECT ST_GeomFromText(
           'POLYGON((-74.8 42.7, -75.1 42.7,
            -75.1 42.6, -74.8 42.7))', 4326);

SELECT ST_GeomFromText(
           'MULTIPOINT(-74.9 42.7, -75.1 42.7)',
           4326);

SELECT ST_GeomFromText(
           'MULTILINESTRING((-76.27 43.1,
           -76.06 43.08), (-76.2 43.3, -76.2 43.3,
           -76.4, 43.1))', 4326);

SELECT ST_GeomFromText(
           'MULTIPOLYGON(((-74.92 42.7, -75.06 42.71,
           -75.07 42.64, -74.92 42.7), (-75.0 42.66,
           -75.0 42.64, -74.98 42.64, -74.98 42.66,
           -75.0 42.66)))', 4326);
```

For each example, we give a WKT string as the first input & the SRID `4326` as the second. In the first example, we create a Point by inserting the WKT `POINT` string as the first argument to `ST_GeomFromText()` with the SRID as the optional second argument. We use the same format in the rest of the examples.

Be sure to mind the number of parentheses that segregate objects, particularly in complex structures such as the MultiPolygon. For example, we need to use two opening parentheses & enclose each polygon's coordinates within another set of parentheses.

If you run each statement separately in pgAdmin, you can view both its data output & visual representation. Upon execution, each statement should return a single column of the `geometry` data type displayed as a string of characters:

<img src = "Using ST_GeomFromText() to Create Spatial Objects.png" width = "600" style = "margin:auto"/>

The string is of the format *extended well-known binary* (EWKB), which you typically won't need to interpret directly. Instead, you'll use columns of geometry (or geography) data as inputs to other functions. To see the visual representation, click the map/brochure-looking icon in the pgAdmin result column header. That should open a Geometry Viewer pane in pgAdmin that displays the geometry atop a map that uses OpenStreetMap as the base layer. For example, the `MULTIPOLYGON` example should look like a triangle & a rectangle.

<img src = "Viewing Geometries in pgAdmin.png" width = "600" style = "margin:auto"/>

## Creating a Geography Type from Well-Known Text

To create a `geography` data type, you can use `ST_GeogFromText(WKT)` to convert a WKT or `ST_GeogFromText(EWKT)` to convert a PostGIS-specific variation called *extended WKT* that includes the SRID. The query shows how to pass in the SRID as part of the extended WKT string to create a MultiPoint `geography` object with three points:

```
SELECT ST_GeogFromText('SRID=4326;MULTIPOINT(
           -74.9 42.7, -76.1 42.7, -74.924 42.6)');
```

Again, you can view the Points on a map by clicking the map/brochure icon in the geography column in the pgAdmin results grid.

<img src = "Using ST_GeogFromText() to Create Spatial Objects.png" width = "600" style = "margin:auto"/>

Along with the all-purpose `ST_GeomFromText()` & `ST_GeogFromText()` functions, PostGIS includes several that are specific to creating certain spatial objects.

## Using Point Functions

The `ST_PointFromText()` & `ST_MakePoint()` functions will turn a WKT `POINT` or a collection of coordinates, respectively, into a `geometry` data type. Points mark coordinates, such as longitude & latitude, which you would use to identify locations or use as building blocks of other objects such as LineStrings.

```
SELECT ST_PointFromText(
           'POINT(-74.9233606 42.699992)', 4326);

SELECT ST_MakePoint(-74.9233606, 42.699992);

SELECT ST_SetSRID(ST_MakePoint(
           -74.9233606, 42.699992), 4326);
```

<img src = "Functions Specific to Making Points.png" width = "600" style = "margin:auto"/>

The `ST_PointFromText(WKT, SRID)` function creates a point `geometry` type from a WKT `POINT` & an optional SRID as the second input. The PostGIS documentation notes that the function includes validation of coordinates that makes it slower than the `ST_GeomFromText()` function.

The `ST_MakePoint(x, y, z, m)` function creates a point `geometry` type on two-, three-, & four-dimensional grid. The first two parameters, `x` & `y` in the example, represent longitude & latitude coordinates. You can use the optional `z` to represent altitude & `m` to represent a measure. That would allow us to, for example, mark a water fountain on a bike trail at a certain altitude & certain distance from the start of the trail. The `ST_MakePoint()` function is faster than `ST_GeomFromText()` & `ST_PointFromText()`, but if you want to specify an SRID, you'll need to designate one by wrapping it inside the `ST_SetSRID()` function.

## Using LineString Functions

Now let's examine some functions we use specifically for creating LineString `geometry` data types. The query below shows how they work.

```
SELECT ST_LineFromText('LINESTRING(-105.90 35.67,
           -105.91 35.67)', 4326);

SELECT ST_MakeLine(ST_MakePoint(-74.9, 42.7),
           ST_MakePoint(-74.1, 42.4));
```

<img src = "Functions Specific to Making LineStrings.png" width = "600" style = "margin:auto"/>

The `ST_LineFromText(WKT, SRID)` function creates a LineString from a WKT `LINESTRING` & an optional SRID as its second input. Like `ST_PointFromText()` earlier, this function includes validation of coordinates that makes it slower than `ST_GeomFromText()`.

The `ST_MakeLine(geom, geom)` function creates a LineString from inputs that must be of the `geometry` data type. The example uses two `ST_MakePoint()` functions as inputs to create the start & endpoint ofthe line. You can also pass in an `ARRAY` object with multiple points, perhaps generated by a subquery, to generate a more complex line.

## Using Polygons Functions

Let's look at three Polygon functions: `ST_PolygonFromText()`, `ST_MakePolygon()`, & `ST_MPolyFromText()`. All create `geometry` data types. The query below shows how you can create Polygons with each.

```
SELECT ST_PolygonFromText('POLYGON((-74.9 42.7,
           -75.1 42.7, -75.1 42.6, -74.9 42.7))', 4326);

SELECT ST_MakePolygon(ST_GeomFromText('LINESTRING(
           -74.92 42.7, -75.06 42.71, -75.07 42.64,
           -74.92 42.7)', 4326));

SELECT ST_MPolyFromText('MULTIPOLYGON(((-74.92 42.7,
           -75.06 42.71, -75.07 42.64, -74.92 42.7),
           (-75.0 42.66, -75.0 42.64, -74.98 42.64,
           -74.98 42.66, -75.0 42.66)))', 4326);
```

<img src = "Functions Specific to Making Polygons.png" width = "600" style = "margin:auto"/>

The `ST_PolygonFromText(WKT, SRID)` function creates a Polygon from a WKT `POLYGON` & an optional SRID. As with the similarly named functions for creating points & lines, it includes a validation step that makes it slower than `ST_GeomFromText()`.

The `ST_MakePolygon(linestring)` function creates a Polygon from a LineString that must open & close with the same coordinates, ensuring the object is closed. This example uses `ST_GeomFromText()` to create the LineString geometry using a WKT `LINESTRING`.

The `ST_MPolyFromText(WKT, SRID)` function creates a MultiPolygon from an WKT & an optional SRID.

Now you have the building blocks to analyse spatial data.

---

# Analysing Farmer's Markets Data

The National Farmers' Market Directory from the US Department of Agriculture catalogs the location & offers of more than 8,600 "markets that feature two or more farm vendors selling agricultural products directly to customers at a common, recurrent physical location".

The *farmers_markets.csv* file contains a portion of the USDA data on each market. Create & load a `farmers_markets` table.

```
CREATE TABLE farmers_markets (
    fmid bigint PRIMARY KEY,
    market_name text NOT NULL,
    street text,
    city text,
    county text,
    st text NOT NULL,
    zip text,
    longitude numeric(10, 7),
    latitude numeric(10, 7),
    organic text NOT NULL
);

COPY farmers_markets
FROM '/YourDirectory/farmers_markets.csv'
WITH (FORMAT CSV, HEADER);
```

The table contains routine address data plus the `longitude` & `latitude` for most markets. An `organic` column indicates whether the market offers organic products; a hyphen (-) in that column indicates an unknown value. After you import the data, count the rows using `SELECT count(*) FROM farmers_markets;` If everything is imported correctly, you should have 8,681 rows.

<img src = "Creating & Loading the farmers_markets Table.png" width = "600" style = "margin:auto"/>

## Creating & Filling a Geography Column

To perform spatial queries on the markets' longitude & latitude, we need to convert those coordinates into a single column with a spatial data type. Because we're working with locations spanning the entire United States & an accurate measurement of a large spherical distance is important, we'll use the `geography` type. After creating the column, we can update it using Points derived from the coordinates & then apply an index to speed up queries.

```
ALTER TABLE farmers_markets
ADD COLUMN geog_point geography (POINT, 4326);

UPDATE farmers_markets
SET geog_point = ST_SetSRID(ST_MakePoint(
        longitude, latitude)::geography, 4326);

CREATE INDEX market_pts_idx ON farmers_markets
USING GIST (geog_point);

SELECT longitude, latitude, geog_point,
       ST_AsEWKT(geog_point)
FROM farmers_markets
WHERE longitude IS NOT NULL
LIMIT 5;
```

The `ALTER TABLE` statement with the `ADD COLUMN` option creates a column of the `geography` type called `geog_point` that will hold points & reference the WGS 84 coordinate system, which we denote using SRID `4326`.

Next, we run a standard `UPDATE` statement to fill the `geog_point` column. Nested inside an `ST_SetSRID()` function, the `ST_MakePoint()` function takes as input the `longitude` & `latitude` columns from the table. The output, which is the `geometry` type of default, must be case to `geography` to match the `geog_point` column type. To do this, we add the PostgreSQL-specific double-colon syntax (::) to the output of `ST_MakePoint()`.

## Adding a Spatial Index

We add an index to the new column to speed up queries. PostgreSQL's default index is the B-tree. A B-tree index is useful for data that you can order & search using equality & range oeprators, but it's less useful for spatial objects. The reason is that you cannot easily sort GIS data along one axis. For example, the application has no way to determine which of these coordinate pairs is greatest: (0, 0), (0, 1) or (1, 0).

Instead, the makers of PostGIS include support for an index designed for spatial data called *R-tree*. In an R-tree index, each spatial item is represented in the index as a rectangle that surrounds its boundaires, & the index itself is a hierarchy of rectangles.

We add a spatial index to the `geog_point` column by including the keywords `USING GIST` in the `CREATE INDEX` statement. `GIST` refers to a generalised searched tree (GiST), an interface to facilitate incorporating specialised indexes to the database.

With the index in place, we use the `SELECT` statement to view the geography data to show the newly encoded `geog_points` column. To view the extended WKT version of `geog_point`, we wrap it in a `ST_AsEWKT()` function to show the extended well-known text coordinates & SRID. The results should look similar to this, with `geog_point` truncated for brevity:

<img src = "Creating & Indexing a geography Column.png" width = "600" style = "margin:auto"/>

<img src = "Creating & Indexing a geography Column Geography View.png" width = "600" style = "margin:auto"/>

## Finding Geographies Within a Given Distance

There is a massive Downtown Farmers' Market in Des Moines, Iowa. With hundreds of vendors, the market spanned several city blocks in the Iowa capital. Farming is big business there, & even though the downtown market is huge, it's not the only one in the area. Let's use PostGIS to find more farmers' markets near downtwon Des Moines.

The PostGIS function `ST_DWithin()` returns a Boolean value of `true` if one spatial object is within a specified distance of another object. If you're working with the `geography` data type, as we are here, you need to use meters as the distance unit. If you're using the `geometry` type, use the distance unit specified by the SRID.

The query below uses the `ST_DWithin()` function to filter `farmers_markets` to show markets within 10 kilometers of the Downtown Farmers' Market in Des Moines.

```
SELECT market_name, city, st, geog_point
FROM farmers_markets
WHERE ST_DWithin(geog_point, ST_GeogFromText(
		  'POINT(-93.6204386 41.5853202)'), 10000)
ORDER BY market_name;
```

The first inputfor `ST_DWithin()` is `geog_point`, which holds the location of each row's market in the `geography` data type. The second input is the `ST_GeogFromText()` function that returns a Point geography from WKT. The coordinates `-93.6204386` & `41.5853202` represent the longitude & latitude of the Downtown Farmers' Market. The final input is `10000`, which is the number of meters in 10 kilometers. The database calculates the distance between each market in the table & the downtown market. If a market within 10 kilometers, it is included in the results.

We're using Points here, but this function works with any geography or geometry type. If you're working with objects such as polygons, you can use the related `ST_DFullyWithin()` function to find objects that are completely within a specified distance.

The result should show nine rows:

<img src = "Using ST_DWithin() to Locate Farmers' Markets within 10 KM of a Point.png" width = "600" style = "margin:auto"/>

One of these nine markets is the Downtown Farmers' Market in Des Moines, which makes sense because its location is at the point used for comparison. The rest are other markets in Des Moines or in nearby West Des Moines.

To see these points on a map, in pgAdmin's results grid, click the eye icon in the `geog_point` column header. The geography viewer should display a map as shown.

<img src = "Farmer's Markets Near Downtown Des Moines, Iowa.png" width = "600" style = "margin:auto"/>

This operation should be familiar; it's a standard feature on many online maps & product apps that lets you locate stores or points of interest near you. Although this list of nearby markets is helpful, it would be even better to know the exact distance of markets from downtown.

## Finding the Distance Between Geographies

The `ST_Distance()` function returns the minimum distance between two geometries, providing meters for geographies & SRID units for geometries. For example, the query below finds the distance in miles from Yankee Stadium in New York City's Bronx borough to Citi Field in Queens, home of the New York Mets.

```
SELECT ST_Distance(ST_GeogFromText('POINT(
           -73.9283685 40.8296466)'),
           ST_GeogFromText('POINT(-73.9283685
           40.7570917)')) / 1609.344
           AS mets_to_yanks;
```

To convert the distance units from meters to miles, we divide the result of `ST_Distance()` by 1609.344 (the number of meters in a mile). The result is about 6.5 miles.

<img src = "Using ST_Distance() to Calculate Distance Between Yankee Stadium & Citi Field.png" width = "600" style = "margin:auto"/>

Let's apply this technique to the farmers' market data using the code below. We'll again find all farmer's markets within 10 kilometers of the Downtown Farmers' Market in Des Moines & show the distance in miles.

```
SELECT market_name, city,
       round((ST_Distance(geog_point, ST_Geog_FromText(
       'POINT(-93.6204386 41.5853202)')) /
       1609.344)::numeric, 2) AS miles_from_dt
FROM farmers_markets
WHERE ST_DWithin(geog_point(ST_GeogFromText('POINT(
          -93.6204386 41.5853202)', 10000)
ORDER BY miles_from_dt ASC;
```

The query uses `ST_Distance()` function as a column to calculate & display the distance from downtown. It's wrapped inside `round()` to trim the output. We provide `ST_Distance()` with the same two inputs we gave `ST_DWithin()`: `geog_point` & the `ST_GeogFromText()` function. The `ST_Distance()` function then calculates the distance between the points specified by both inputs, returning the result in meters. To convert to miles, we divide by `1609.344`, the approximate number of meters in a mile. Then, to provide the `round()` function with the correct input data type, we case the column result to type `numeric`.

The `WHERE` clause uses the same `ST_DWithin()` function & inputs as in `ST_Distance()`. You should see the following results, ordered by distance in ascending order:

<img src = "Using ST_Distance() For Each Row in farmers_markets.png" width = "600" style = "margin:auto"/>

You see this type of result often when searching online for a store or address. You might also find this technique useful for other analysis scenarios, such as finding all the schools within a certain distance of a known source of pollution or all the homes within five miles of an airport.

## Finding the Nearest Geographies

Sometimes it's helpful to have the database simply reteurn the spatial objects that are in closest proximity to another object without specifying some arbitrary distance in which to search. For example, we may want to find the closest farmers' market regardless of whether it's 10 kilometers away or 100. To do that, we can instruct PostGIS to implement a *K-nearest-neighbours* search algorithm by using the `<->` distance operator in the `ORDER BY ` clause of the query. Nearest neighbours algorithms solve a range of classification problems by identifying similar items -- text recognition is an example. In this case, PostGIS will identify some number of spatial objects, represented by `K`, nearest to an object we specify.

For example, let's say we're planning to visit the vacation spot of Bar Harbor, Maine, & want to find the three farmer's markets closest to town. We can use the query below:

```
SELECT market_name, city, st,
       round((ST_Distance(geog_point, ST_GeogFromText(
           'POINT(-68.2041607 44.3876414)')) /
           1609.344)::numeric, 2) AS miles_from_bh
FROM farmers_markets
ORDER BY geog_point <-> ST_GeogFromText('POINT(
             -68.2041607 44.3876414)')
LIMIT 3;
```

The query uses an `ORDER BY `clause that contains the `<->` distance operator. To the left of the operator, we place the `geog_point` column; to the right we supply the WKT for the Point locating downtown Bar Harbor inside `ST_GeogFromText()`. In effect, this syntax says, "Order the results by distance from the geography to the Point". 

Adding `LIMIT 3` restricts the results to the three closest markets (the three nearest neighbours):

<img src = "Using the <-> Distance Operator for a Nearest Neighbours Search.png" width = "600" style = "margin:auto"/>

You can, of course, change the number in the `LIMIT` clause to return more or fewer results. Using `LIMIT 1`, for example, will return only the closest market.

---

# Working with Census Shapefiles

A *shapefile* is a GIS data file format developed by Esri, a US company known for its ArcGIS mapping visualisation & analysis platform. Shapefiles are a standard file format for GIS platforms -- such as ArcGIS & the open source QGIS -- & are used by governments, corporations, nonprofits, & technical organisations to display, analyse, & distribute data with geographic features.

Shapefiles hold information describing the shape of a feature (such as a county, a road, or a lake) plus a database with each feature's attributes. Those attributes might include their name & other demographic descriptors. A single shapefile can contain only one type of shape, such as polygons or points, & when you load a shapefile into a GIS platform that supports visualisation, you can view the shapes & query their attributes. PostgreSQL, with the PostGIS extension, lets you query the spatial data in the shapefile.

## Understanding the Contents of a Shapefile

A shapefile comprises a collection of files with different extensions, each with a different purpose. Often, when you download a shapefile, it comes in a compressed archive, such as *.zip*. You'll need to unzip it to access the individual files.

Per ArcGIS documentation, these are the most common extensions you'll encounter:

* **.shp:** Main file that stores the feature geometry.
* **.shx:** Index file that stores the index of the feature geometry.
* **.dbf:** Database table (in dBASE format) that stores the attribute information of features.
* **.xml:** XML-format file that stores metadata about the shapefile.
* **.prj:** Projection file that stores the coordinate system information. You can open this file with a text editor to view the geographic coordinate system & projection.

According to the documentation, files with the first three extensions include necessary data required for working with a shapefile. The other file types are optional. You can load a shapefile into PostGIS to access its spatial objects & attributes for each. 

*tl_2019_us_county.zip* from the lesson's materials contains files including the extensions listed above. Unzip it.

## Loading Shapefile

For macOS, we'll use the command line application `shp2pgsql`.

### Importing Shapefiles using shp2pgsql

The Shapefile Import/Export Manager isn't available on all PostGIS distributions for macOS & Linux. For this reason, we'll use the PostGIS commad line tool `shp2pgsql`, which lets you accomplish the same thing using a single text command.

On macOS & Linux, we execute command line tools in the Terminal application. At the command line, you use the following syntax to import a shapefile into a new table:

```
shp2pgsql -I -s SRID -W encoding shapefile_name table_name |
psql -d database -U user
```

A lot's happening here. Let's look at each argument following the command:

* `-I` adds an index on the new table's geometry column using GiST.
* `-s` lets you specify an SRID for the geometric data.
* `-W` lets you specify file encoding, if necessary.
* `shape_file_name` is the name (including full path) of the file ending with the *.shp* extension.
* `table_name` indicates the next table you want the shapefile imported to.

Following these arguments, you place a  pipe symbol (`|`) to direct the output of `shp2pgsql` to `psql`, the PostgreSQL command line utility. That's followed by arguments for naming the database & user. For example, to load the *tl_2019_us_county.shp* shapefile from the lesson's resources into a `us_counties_2019_shp` table in the `analysis` database, in your terminal you would move to the directory containing the shapefile & run the following command (all on one line):

```
shp2pgsql -I -s 4269 -W LATIN1 tl_2019_us_county.shp
us_counties_2019_shp | psql -d analysis -U postgres
```

The server should respond with a number of SQL `INSERT` statements before creating the index & returning you to the command line. It might take some time to construct the entire set of arguments the first time around. But after you've done one, subsequent imports should take less time because you can imply substitute file & table names into the syntax you already wrote.

## Exploring the Census 2019 Counties Shapefile

Your new `us_counties_2019_shp` table contains columns including each county's name as well as the *Federal Information Processing Standards* (FIPS) codes uniquely assigned to each state & county. The `geom` column contains the spatial data for each county's boundary. To start, let's check what kind of spatial object `geom` contains using the `ST_AsText()` function. The query below shows the WKT representation of the first `geom` value in the table.

```
SELECT ST_AsText(geom)
FROM us_counties_2019_shp
ORDER BY gid
LIMIT 1;
```

The result is a MultiPolygon with hundreds of coordinate pairs. 

<img src = "Checking the geom Column's WKT Representation.png" width = "600" style = "margin:auto"/>

Each coordinate pair marks a Point on the boundary of the county, & remember that a `MULTIPOLYGON` object can contain a set of polygons. In the case of US counties, that will enable storage of counties whose boundaries contain more than one distinct, separated area.

### Finding the Largest Counties in Square Miles

Which county can claim the title of largest in area? To find the answer, we use the `ST_Area()` function, which returns the area of a Polygon or MultiPolygon object. If you're working with a `geography` data type, `ST_Area()` returns the result in square meters. With a `geography` data type, `ST_Area()` returns the result in square meters. With a `geometry` data type -- as used with this shapefile -- the function returns the area in SRID units. Typically, those units are not useful for practical analysis, so we'll cast the `geometry` type to `geography` to obtain square meters. It's an intensive calculation, so expect extra time for this query to complete.

```
SELECT name,
       statefp AS st,
       round((ST_Area(geom::geography) /
           2589988.110336)::numeric, 2)
           AS square_miles
FROM us_counties_2019_shp
ORDER BY square_miles DESC
LIMIT 5;
```

The `geom` column is data type `geometry`, so to find the area in square meters, we cast the `geom` column to a `geography` data type using the double-colon syntax. Then, to get square miles, we divide the area by 2589988.110336, which is the number of square meters in a square mile. To make the result easier to read, we wrap it in the `round()` function & name the resulting column `square_miles`. Finally, we list the results in descending order from the largest area to the smallest & use `LIMIT 5` to show the first five results, which should look lik this:

<img src = "Finding the Largest Counties by Area using ST_Area().png" width = "600" style = "margin:auto"/>

Congratulations to Alaska, where the boroughs (the name for county equivalents) are big. The five largest are all in Alaska, denoted by the state FIPS code `02`. Yukon-Koyukuk, located in the heart of Alaska, is more than 147,800 square miles.

### Finding a County by Longitude & Latitude

If you've ever wondered how spammy online ads seem to know where you live, it's thanks to *geolocation services* that use various means, such as your phone's GPS, to find your longitude & latitude. Given your coordinates, a spatial query can then determine which geography (city or town, for example) that point falls into.

You can replicate this technique using your census shapefile & the `ST_Within()` function, which returns `true` if one geometry is inside another on the coordinate grid. The query below shows an example using the longitude & latitude of downtown Hollywood, California:

```
SELECT sh.name,
       c.state_name
FROM us_counties_2019_shp AS sh
JOIN us_counties_pop_est_2019 AS c
ON sh.statefp = c.state_fips
    AND sh.countyfp = c.county_fips
WHERE ST_Within('SRID = 4269;POINT(-118.3419063
          34.0977076)'::geometry, geom);
```

The `ST_Within()` function inside the `WHERE` clause requires two `geometry` inputs & evaluates whether the first is inside the second. For the function to work properly, both `geometry` inputs must have the same SRID. In this example, the first input is an extended WKT representation of a Point that includes the SRID `4269` (same as the census data), which is cast as a `geometry` type. The `ST_Within()` function doesn't accept a separate SRID input, so to set it for the supplied WKT, you must prefix it to the string like this: `'SRID=4269;POINT(-118.3419063 34.0977076)'`. The second input is the `geom` column from the table.

You should see the following result:

<img src = "Using ST_Within() to Find the County Belonging to a Pair of Coordinates.png" width = "600" style = "margin:auto"/>

It shows that the Point we supplied is within Los Angeles county in California. We also see how this technique can gain value (or raise privacy concerns) by relating a point to data about its surrounding area -- as we did here by joining to county population estimates. Suddenly, we can tell a lot about someone based on data describing where they spend time.

## Examining Demographics Within a Distance

A fundamental metric for planners trying to locate a new school, business, or other community amenity is the number of people who live within a certain distance of it. Will there be enough people nearby to make construction worthwhile? To find the answer, we can use spatial & demographics data to estimate the population contained in the geographies within a certain distance of the planned location.

Say we're considering building a restaurant in downtown Lincoln, Nebraska, & we want to understand how many people live with 50 miles of the potential location. The query below uses the `ST_DWithin()` function to find counties that have any portion of their boundary within 50 miles of downtown Lincoln & sum their estimated 2019 population:

```
SELECT sum(c.pop_est_2019) AS pop_est_2019
FROM us_counties_2019_shp AS sh
JOIN us_counties_pop_est_2019 AS c
ON sh.statefp = c.state_fips
    AND sh.countyfp = c.county_fips
WHERE ST_DWithin(sh.geom::geography,
    ST_GeogFromText('SRID=4269;POINT(-96.699656
    40.811567)'), 80467);
```

We used `ST_DWithin()` to find farmers' markets close to Des Moines, Iowa. Here, we apply the same technique. We pass three arguments to `ST_DWithin()`: the census shapefile's' `geom` column case to the `geography` type; a point representing downtown Lincoln; & the distance of 50 miles using its equivalent in meters, 80,467.

The query should return a sum of 1,470,295, using the data from the joined census estimates table's `pop_est_2019` column:

<img src = "Using ST_DWithin() to Count People Near Lincoln, Nebraska.png" width = "600" style = "margin:auto"/>

Say we want to list the county names & visualise their borders in pgAdmin; we can modify our query:

```
SELECT sh.name,
       c.state_name,
       c.pop_est_2019,
       ST_Transform(sh.geom, 4326) AS geom
FROM us_counties_2019_shp AS sh
JOIN us_counties_pop_est_2019 AS c
ON sh.statefp = c.state_fips
    AND sh.countyfp = c.county_fips
WHERE ST_DWithin(geom::geography, ST_GeogFromText(
          'SRID=4269;POINT(-96.699656 40.811567)'),
          80467);
```

This query should return 25 rows with the county name & its population. If you click the map/brochure icon in the header of the `geom` column, you should see the counties displayed on a map in pgAdmin's Geometry Viewer:

<img src = "Counties That Have a Potion of Their Boundaries within 50 Miles of Lincoln.png" width = "600" style = "margin:auto"/>

The queries show counties that have any portion of their boundaries within 50 miles of Lincoln. Because counties tend to be large in area, they're a bit crude for determining an exact number of people within the distance of the point. For a more precise count, we could use smaller census geographies such as tracts or block groups, both of which are subcomponents of counties.

Finally, note that the pgAdmin Geometry Viewer's base map is the free OpenStreetMap, which uses the WGS 84 coordinate system. Our census shapefiles use a different coordinate system: North American Datum 83. For our data to display properly against the base map, we must use the `ST_Transform()` function to convert the census geometry to the SRID of 4326. If we omit that function, the geographies will display on a blank canvas in the viewer because the coordinate systems don't match.

---

# Performing Spatial Joins

Joining tables with spatial data opens up interesting opportunities for analysis. For example, you can join a table of coffee shops (which include longitude & latitude) to the counties table to find out how many shops exist in each county based on their location. 

## Exploring Roads & Waterways Data

Much of the year, the Santa Fe River, which cuts through the New Mexico state capital, is a dry riverbed better described as an *intermittent stream*. According to the Santa Fe city website, the river is susceptible to flash flooding & was named the nation's most endangered river in 2007. If you were an urban planner, it would help to know where the river intersects roadways so you could plan for emergency response when it floods. 

You can find these locations using another set of US Census TIGER/Line shapefiles that has details on roads & waterways in Santa Fe County. These shapefiles are also included with the lesson's resources. Unzip *tl_2019_35049_linearwater.zip* & *tl_2019_35049_roads.zip*, & then import both using the same steps from earlier. Name the water table `santafe_linearwater_2019` & the roads table `santafe_roads_2019`.

```
shp2pgsql -I -s 4269 -W LATIN1 tl_2019_35049_linearwater.shp
santafe_linearwater_2019 | psql -d analysis -U postgres

shp2pgsql -I -s 4269 -W LATIN1 tl_2019_35049_roads.shp
santafe_roads_2019 | psql -d analysis -U postgres
```

Next, refresh your database & run a quick `SELECT * FROM` query on both tables to view the data, you should have 11,655 rows in the roads table & 1,148 in the linear water table.

<img src = "Creating the santafe_roads_2019 Table.png" width = "600" style = "margin:auto"/>

<img src = "Creating the santafe_linearwater_2019 Table.png" width = "600" style = "margin:auto"/>

As with the counties shapefile, both tables have an indexed `geom` column of type `geometry`. It's helpful to check the type of spatial object in the column, so you know the type of spatial feature you're querying. You can do that using the `ST_AsText()` function or `ST_GeometryType()`, as shown in the query below:

```
SELECT ST_GeometryType(geom)
FROM santafe_linearwater_2019
LIMIT 1;

SELECT ST_GeometryType(geom)
FROM santafe_roads_2019
LIMIT 1;
```

Both queries should return one row with the same value: `ST_MultiLineString`. That tells us that waterways & roads are stored as MultiLineString objects, a set of LineStrings that can be noncontinuous.

<img src = "Using ST_GeometryType() to Determine Geometry.png" width = "600" style = "margin:auto"/>

## Joining the Census Roads & Water Tables

To find all the roads in Santa Fe that intersect the Santa Fe River, we'll join the roads & waterway tables with a query that tells us where the objects touch. We'll do this by using the `ST_Intersects()` function, which returns a Boolean `true` if two spatial objects contact each other. Inputs can be either `geometry` or `geography` types.

```
SELECT water.fullname AS waterway,
       roads.rttyp,
       roads.fullname AS road
FROM santafe_linearwater_2019 AS water
JOIN santafe_roads_2019 AS roads
    ON ST_Intersects(water.geom, roads.geom)
WHERE water.fullname = 'Santa Fe Riv'
    AND roads.fullname IS NOT NULL
ORDER BY roads.fullname;
```

The `SELECT` column list includes the `fullname` column from the `santafe_linearwater_2019` table, which gets `water` as its alias in the `FROM` clause. The column list includes the `rttyp` code, which represents the route type, & `fullname` columns from the `santafe_roads_2019` table, aliased as `roads`.

In the `ON` portion of the `JOIN` construct, we use the `ST_Intersects()` function with the `geom` column from both tables as inputs. Here, the expression evaluates as `true` if the geometries intersect. We use `fullname` to filter the results to show only those that have the full string `'Santa Fe Riv'`, which is how the Santa Fe River is listed in the water table. We also eliminate instances where road names are `NULL`. The query should return 37 rows.

<img src = "Spatial Join with ST_Intersects() to Find Roads Crossing the Santa Fe River.png" width = "600" style = "margin:auto"/>

Each road in the results intersects with a portion of the Santa Fe River. The route type code for each of the first results is `M`, which indicates that the road name shown is its *common* name as opposed to a county or state recognised name, for example. Other road names in the complete results carry rout types of `C`, `S`, `U` (for unknown). The full route type code list is available [here](https://www.census.gov/library/reference/code-lists/route-type-codes.html).

## Finding the Location Where Objects Intersect

We successfully identified roads that intersect the Santa Fe River. Although that's great, it would really help to know the precise location of each intersection. We can modify the query to include the `ST_Intersection()` function, which returns the location of the place where objects touch.

```
SELECT water.fullname AS waterway,
       roads.rttyp,
       roads.fullname AS road,
       ST_AsText(ST_Intersection(water.geom, roads.geom))
FROM santafe_linearwater_2019 AS water
JOIN santafe_roads_2019 AS roads
    ON ST_Intersects(water.geom, roads.geom)
WHERE water.fullname = 'Santa Fe Riv'
    AND roads.fullname IS NOT NULL
ORDER BY roads.fullname;
```

The function returns a geometry object, so to view its WKT representation, we must wrap it in `ST_AsText()`. The `ST_Intersection()` function takes two inputs: the `geom` columns from both the `water` & `roads` tables. The results should now include the exact coordinate location, or locations, where the river crosses the roads.

<img src = "Using ST_Intersection() to Show Where Roads Cross the River.png" width = "600" style = "margin:auto"/>

This is much better than poring over a map with a pencil, & might prompt more ideas for analysing spatial data. For example, if you have a shapefile of building footprints, you could find buildings near the river & in danger of flooding during heavy rains. Governments & private organisations regularly use these techniques as part of their planning process.

---

# Wrapping Up

Mapping is a powerful analysis tool, & the techniques learned in this lesson give you a strong start to exploring with PostGIS. To visualise this data, you may want to use a GIS application such as ArcGIS or the open source QGIS. Both can use PostGIS-enabled PostgreSQL database as a data source, allow you to visualise shapefile data in your tables or the results of queries.